In [1]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta
from datetime import datetime



## Load Task Duration Config
For each task type that could be generated we need an average duration and a maximum duration.

In [2]:
#Load task duration dataframe
#task_duration_df = pd.read_csv('task_duration_config.csv')
#task_duration_df.set_index('Task', inplace=True)

data = [['New Client Onboarding Request', 0.5,1.5], 
        ['Review Documents', 0.25,  2.5], 
        ['Automated Scoreboarding', 0.1, 0.15],
        ['Manual Scoreboarding', 1.0, 3.0],
        ['Update Backend Systems', 0.25, 0.5],
        ['Notification Review Request Completed', 0.1, 0.15]
       ]

task_duration_df = pd.DataFrame(data, columns=['Task', 'Avg', 'Max'])
task_duration_df.set_index('Task', inplace=True)

In [3]:

#task_duration_df = pd.read_csv('task_duration_config.csv')
#task_duration_df.set_index('Task', inplace=True)

In [4]:
task_duration_df.head(10)

,Avg,Max
Task,,
New Client Onboarding Request,0.50,1.50
Review Documents,0.25,2.50
Automated Scoreboarding,0.10,0.15
Manual Scoreboarding,1.00,3.00
Update Backend Systems,0.25,0.50
Notification Review Request Completed,0.10,0.15


## Utility Functions

### Generate Process Instances
This function manages the creation of process instances for a specific process variant. A process variant is simply the list of tasks in execution order, including any loops. For example ['Wake Up', 'Breakfast', 'Work', 'Dinner' 'Sleep']

In [5]:
def generate_process_instances(process_variant, qty):

    global start_date_time
    global instance_counter
    task_list = []
    for x in range(0, qty):
        tasks = build_task_list(instance_counter, process_variant, start_date_time, task_duration_df)
        for task in tasks:
            task_list.append(task)

        # Increment the start time by 24 hour
        start_date_time = start_date_time + timedelta(hours=24)
        # Increment the process_id
        instance_counter = instance_counter +1
        
    return task_list

## Build Task List
The build_task_list function creates a list of tasks for a specific instance of a process. The sequence of tasks is defined in the variable called process_variant. Random task durations are calculated using the task_durations_df that contains the task name, the average duration and the max duration.  

This code generates a random duration by leveraging the numpy lognormal function that gives a random
number drawn from a log normal distribution. Look up a picture of log normal distributions and you'll see 
why this is useful for generating random durations based on a mean and std deviation.
most samples are near the average with a long tail stretching towards infinity.


In [6]:
def build_task_list(instance_id, process_variant, start_date_time, task_duration_df):
    instance_task_list = []
    rnd = np.random.default_rng()  # a random number generator

    # process_variant is a series
    for task_type in process_variant:
        # get the avg and max durations from the task_df dataframe using task_type as the key
        avg_dur = task_duration_df.loc[task_type, 'Avg']
        max_dur = task_duration_df.loc[task_type, 'Max']

        sigma = (max_dur - avg_dur) / max_dur  # std dev
        log_mean = np.log(avg_dur)  # can't pass the mean duration into lognormal until it has been logged itself 
        delta = rnd.lognormal(log_mean, sigma)  # get a random sample from a log normal distribution with a std dev

        task = [instance_id, task_type, start_date_time]
        instance_task_list.append(task)

        # increment the start time by the delta so that the next task start after this one
        start_date_time = start_date_time + timedelta(hours=delta)

    return instance_task_list

## Shift Activity Start Time
This function shifts the activity start time for a specific task with a specific attribute set to a specific value. For example, to delay the start time of breakfast by 1hour "task_list, User, Gerry, Eat Breakfast, 1"

This function will apply the same timeshift to every subsequent task in the proccess instance. 

In [7]:
def shiftActivityStartTime(tasks, target_attribute, target_value, target_task, timeShift):

    # Logic requires we sort by process_id and start_date
    tasks = tasks.sort_values(['process_id', 'start_time'])
    tasks = tasks.reset_index(drop=True)
    process_being_modified = -1

    # iterate through the dataframe using the index value
    for x in tasks.index:
        row = tasks.loc[x]
        task_attribute = row[target_attribute]
        row_task = row['task']
        current_process = row['process_id']


        # Have we found the target activity
        if row_task == target_task and task_attribute == target_value:
            process_being_modified = current_process

        # Are we still processing the same process that we were when we found the target activity ?
        # If so we are moving all subsequent tasks back by the value of the timeshift parameter
        if current_process == process_being_modified:
            # shift the time
            current_ts = row['start_time']
            shifted_ts = current_ts + timedelta(hours=timeShift)
            tasks.loc[x, 'start_time'] = shifted_ts

    return tasks

## Initialise Data Generator

In [8]:
start_date_time = datetime(2017, 11, 28, 18, 00, 00)
instance_time_offset = 24
instance_counter = 1


## Create Happy Path Instances

In [9]:

qty = 900
happy_path = ["New Client Onboarding Request", 
               "Review Documents", 
               "Automated Scoreboarding",
               "Update Backend Systems", 
               "Notification Review Request Completed"]

happy_path_task_list = []
happy_path_task_list = generate_process_instances(happy_path, qty)
happy_path_task_list_df = pd.DataFrame(happy_path_task_list)

# Increment the start time by 24 hour
start_date_time = start_date_time + timedelta(hours=24)

In [10]:
happy_path_task_list_df.head()

,0,1,2
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000
1,1,Review Documents,2017-11-28 18:16:47.080189
2,1,Automated Scoreboarding,2017-11-28 18:38:38.127292
3,1,Update Backend Systems,2017-11-28 18:48:03.275314
4,1,Notification Review Request Completed,2017-11-28 19:01:32.852061


## Create Instances Requiring Manual Scoreboarding

In [11]:

qty = 80
manual_path = ["New Client Onboarding Request", 
               "Review Documents", 
               "Automated Scoreboarding",
               "Manual Scoreboarding",
               "Update Backend Systems", 
               "Notification Review Request Completed"]

manual_path_task_list = []
manual_path_task_list = generate_process_instances(manual_path, qty)
manual_path_task_list_df = pd.DataFrame(manual_path_task_list)

# Increment the start time by 24 hour
start_date_time = start_date_time + timedelta(hours=24)

In [12]:
manual_path_task_list_df.head()

,0,1,2
0,901,New Client Onboarding Request,2020-05-17 18:00:00.000000
1,901,Review Documents,2020-05-17 18:15:51.781755
2,901,Automated Scoreboarding,2020-05-17 19:15:37.058832
3,901,Manual Scoreboarding,2020-05-17 19:23:11.183341
4,901,Update Backend Systems,2020-05-17 20:49:54.076790


## Create Instance that loop

In [13]:
qty = 20
loop_path = ["New Client Onboarding Request", 
               "Review Documents", 
               "Automated Scoreboarding",
               "Manual Scoreboarding",
               "Review Documents",
               "Update Backend Systems", 
               "Notification Review Request Completed"]

loop_path_task_list = []
loop_path_task_list = generate_process_instances(loop_path, qty)
loop_path_task_list_df = pd.DataFrame(loop_path_task_list)

# Increment the start time by 24 hour
start_date_time = start_date_time + timedelta(hours=24)

In [14]:
# Can only concat two dataframes at a time
df = pd.concat([happy_path_task_list_df,manual_path_task_list_df], axis=0)
df = pd.concat([df,loop_path_task_list_df], axis=0)

In [15]:
df.columns = ['process_id', 'task', 'start_time']

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5120 entries, 0 to 139
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   process_id  5120 non-null   int64         
 1   task        5120 non-null   object        
 2   start_time  5120 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 160.0+ KB


In [17]:
# Confirm how many processes
len(df["process_id"].unique())

1000

In [18]:
df.head(10)

,process_id,task,start_time
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000
1,1,Review Documents,2017-11-28 18:16:47.080189
2,1,Automated Scoreboarding,2017-11-28 18:38:38.127292
3,1,Update Backend Systems,2017-11-28 18:48:03.275314
4,1,Notification Review Request Completed,2017-11-28 19:01:32.852061
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000
6,2,Review Documents,2017-11-29 18:26:51.642022
7,2,Automated Scoreboarding,2017-11-29 18:36:14.033560
8,2,Update Backend Systems,2017-11-29 18:43:02.357288
9,2,Notification Review Request Completed,2017-11-29 19:01:06.665109


## Add Task Level Business Data : User

In [19]:

# Add a column for User
df["user"] = ""

In [20]:
def setRandomUser(row):
    match row["task"]:
        case "Review Documents":
            return random.choice(['Rod','Jane','Freddy'])
        case "New Client Onboarding Request":
            return random.choice(['Clive','Francis','Nick','Seb','Tom'])
        case "Manual Scoreboarding":
            return random.choice(['Sharon','Susan', 'Sam'])
        case "Update Backend Systems":
            return "RPA"
        case "Automated Scoreboarding":
            return "SYSTEM"
        case "Notification Review Request Completed":
            return "SYSTEM"
        case _:
            return row["user"]

In [21]:
df["user"] = df.apply(setRandomUser, axis=1)

In [22]:
df.head(20)

,process_id,task,start_time,user
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,Clive
1,1,Review Documents,2017-11-28 18:16:47.080189,Rod
2,1,Automated Scoreboarding,2017-11-28 18:38:38.127292,SYSTEM
3,1,Update Backend Systems,2017-11-28 18:48:03.275314,RPA
4,1,Notification Review Request Completed,2017-11-28 19:01:32.852061,SYSTEM
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,Tom
6,2,Review Documents,2017-11-29 18:26:51.642022,Rod
7,2,Automated Scoreboarding,2017-11-29 18:36:14.033560,SYSTEM
8,2,Update Backend Systems,2017-11-29 18:43:02.357288,RPA
9,2,Notification Review Request Completed,2017-11-29 19:01:06.665109,SYSTEM


## Add Task Level Business Data : UserGroup

In [23]:
# Add a column for UserGroup
df["user_group"] = ""

In [24]:
def setUserGroup(row):
    match row["task"]:
        case "Review Documents":
            return "Operations"
        case "New Client Onboarding Request":
            return "Sales" 
        case "Manual Scoreboarding":
            return "Risk"
        case "Update Backend Systems":
            return "SYSTEM"
        case "Automated Scoreboarding":
            return "SYSTEM"
        case "Notification Review Request Completed":
            return "SYSTEM"
        case _:
            return ""

In [25]:
df["user_group"] = df.apply(setUserGroup, axis=1)

In [26]:
df.head(20)

,process_id,task,start_time,user,user_group
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,Clive,Sales
1,1,Review Documents,2017-11-28 18:16:47.080189,Rod,Operations
2,1,Automated Scoreboarding,2017-11-28 18:38:38.127292,SYSTEM,SYSTEM
3,1,Update Backend Systems,2017-11-28 18:48:03.275314,RPA,SYSTEM
4,1,Notification Review Request Completed,2017-11-28 19:01:32.852061,SYSTEM,SYSTEM
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,Tom,Sales
6,2,Review Documents,2017-11-29 18:26:51.642022,Rod,Operations
7,2,Automated Scoreboarding,2017-11-29 18:36:14.033560,SYSTEM,SYSTEM
8,2,Update Backend Systems,2017-11-29 18:43:02.357288,RPA,SYSTEM
9,2,Notification Review Request Completed,2017-11-29 19:01:06.665109,SYSTEM,SYSTEM


## Add Process Instance Business Data : Industry
Industy won't change during the process so all tasks for a given process ID must have the same value

In [27]:
df["industry"] = ""

In [28]:
process_id_list = df["process_id"].unique()

for process_id in process_id_list:
    industry = random.choice(['Federal','Finance','Healthcare','Insurance','Telecom'])
    df.loc[df["process_id"].eq(process_id), "industry"] = industry
    
                          
    

In [29]:
df.head(20)

,process_id,task,start_time,user,user_group,industry
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,Clive,Sales,Federal
1,1,Review Documents,2017-11-28 18:16:47.080189,Rod,Operations,Federal
2,1,Automated Scoreboarding,2017-11-28 18:38:38.127292,SYSTEM,SYSTEM,Federal
3,1,Update Backend Systems,2017-11-28 18:48:03.275314,RPA,SYSTEM,Federal
4,1,Notification Review Request Completed,2017-11-28 19:01:32.852061,SYSTEM,SYSTEM,Federal
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,Tom,Sales,Federal
6,2,Review Documents,2017-11-29 18:26:51.642022,Rod,Operations,Federal
7,2,Automated Scoreboarding,2017-11-29 18:36:14.033560,SYSTEM,SYSTEM,Federal
8,2,Update Backend Systems,2017-11-29 18:43:02.357288,RPA,SYSTEM,Federal
9,2,Notification Review Request Completed,2017-11-29 19:01:06.665109,SYSTEM,SYSTEM,Federal


## Add Process Instance Business Data : Service Charge
Service charge will be a random choice based on industry.

In [30]:
df["service_charge"] = 0

In [31]:
def getServiceChargeByIndustry(industry):
    match industry:
        case "Federal":
            return random.choice([3000, 6000, 8000])
        case "Finance":
            return random.choice([10000, 12000, 20000])
        case "Healthcare":
            return random.choice([15000, 20000, 25000])
        case "Insurance":
            return 45000
        case "Telecom":
            return 49000
        case _:
            return 64000

In [32]:
last_process = -1
service_charge = 0

process_id_list = df["process_id"].unique()

for process_id in process_id_list:
        # get the tasks for this process instance, use copy() to make it clear to the interpreter we know
        # this is a copy to supress any warnings about updating copies
        process_instance_tasks = df.loc[df.process_id == process_id].copy()
        # Get the fir row out of the task list and get the industry value
        industry = process_instance_tasks.iloc[0]['industry']
        
        # call function to get a semi-random service charge for the industry
        service_charge = getServiceChargeByIndustry(industry)
        
        # set the service charge in every row for this process instance
        df.loc[df.process_id == process_id, 'service_charge'] = service_charge
    


In [33]:
df.head(50)

,process_id,task,start_time,user,user_group,industry,service_charge
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,Clive,Sales,Federal,3000
1,1,Review Documents,2017-11-28 18:16:47.080189,Rod,Operations,Federal,3000
2,1,Automated Scoreboarding,2017-11-28 18:38:38.127292,SYSTEM,SYSTEM,Federal,3000
3,1,Update Backend Systems,2017-11-28 18:48:03.275314,RPA,SYSTEM,Federal,3000
4,1,Notification Review Request Completed,2017-11-28 19:01:32.852061,SYSTEM,SYSTEM,Federal,3000
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,Tom,Sales,Federal,3000
6,2,Review Documents,2017-11-29 18:26:51.642022,Rod,Operations,Federal,3000
7,2,Automated Scoreboarding,2017-11-29 18:36:14.033560,SYSTEM,SYSTEM,Federal,3000
8,2,Update Backend Systems,2017-11-29 18:43:02.357288,RPA,SYSTEM,Federal,3000
9,2,Notification Review Request Completed,2017-11-29 19:01:06.665109,SYSTEM,SYSTEM,Federal,3000


# Stories
Make a copy of the dataframe so we can restart here if required.

In [38]:
story_df = df.copy()

## Story : 1 - Rod is Too Busy
Rod is always very busy doing other work, so he may not be able to start right away.
Lets take a look at two `Review Documents` tasks that have been done by Rod. 

In [39]:
filter = (story_df['task'] == 'Review Documents') & (story_df['user'] == 'Rod')
story_df[filter].head(2)

,process_id,task,start_time,user,user_group,industry,service_charge
1,1,Review Documents,2017-11-28 18:16:47.080189,Rod,Operations,Federal,3000
6,2,Review Documents,2017-11-29 18:26:51.642022,Rod,Operations,Federal,3000


Delay the start time of Rod's `Review Documents` tasks by 1 hour

In [40]:
# If user is Rod and activity is `Review Documents` shift the start_time by 1 hour
# 
story_df = shiftActivityStartTime(story_df, 'user','Rod', 'Review Documents', 1)

Lets see the result of this change:

In [41]:
filter = (story_df['task'] == 'Review Documents') & (story_df['user'] == 'Rod')
story_df[filter].head(2)

,process_id,task,start_time,user,user_group,industry,service_charge
1,1,Review Documents,2017-11-28 19:16:47.080189,Rod,Operations,Federal,3000
6,2,Review Documents,2017-11-29 19:26:51.642022,Rod,Operations,Federal,3000


## Story : 2 - Complex Doc Review
Increase the time taken for automated scoreboarding to start by 2hrs when the industry is federal. 
The rationale here is that the documentation requirements are greater so the 
previous task `Review Documents` takes much longer.

Lets look at some `Automated Scoreboarding` where the industry is `Federal`

In [44]:
filter = (story_df['task'] == 'Automated Scoreboarding') & (story_df['industry'] == 'Federal')
story_df[filter].head(2)

,process_id,task,start_time,user,user_group,industry,service_charge
2,1,Automated Scoreboarding,2017-11-28 19:38:38.127292,SYSTEM,SYSTEM,Federal,3000
7,2,Automated Scoreboarding,2017-11-29 19:36:14.033560,SYSTEM,SYSTEM,Federal,3000


In [47]:
story_df = shiftActivityStartTime(story_df, 'industry','Federal', 'Automated Scoreboarding', 2)

In [48]:
filter = (story_df['task'] == 'Automated Scoreboarding') & (story_df['industry'] == 'Federal')
story_df[filter].head(2)

,process_id,task,start_time,user,user_group,industry,service_charge
2,1,Automated Scoreboarding,2017-11-28 21:38:38.127292,SYSTEM,SYSTEM,Federal,3000
7,2,Automated Scoreboarding,2017-11-29 21:36:14.033560,SYSTEM,SYSTEM,Federal,3000


## Export Finished Dataset

In [49]:
story_df.to_csv('pm_tasks_1k.csv', index=False, date_format='%d-%m-%Y %H:%M:%S')